In [1]:
%load_ext autoreload
%autoreload

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from mirisim.config_parser import SimulatorConfig
from mirisim import MiriSimulation
import tso_lrs_sims_setup
from tso_lrs_sims_setup import wasp62_scene, wasp62_sim_config

from importlib import reload

In this notebook I'm going to generate simulated MIRI time series LRS (slitless) data, for data management & pipeline testing purposes. To install Mirisim, see the [the public release webpage](http://miri.ster.kuleuven.be/bin/view/Public/MIRISim_Public). The target for the mock observations is WASP-62, an exoplanet host star with the following properties from [the exoplanet encyclopaedia](http://exoplanet.eu/catalog/wasp-62_b/):

* spectral type F7
* T_bb = 6230 K
* K = 8.994

K magnitude of 8.994 corresponds to a flux of 20e3 microJy at 8 micron (value obtained from the ETC), and I'll use this value for the source generation.

Using the ETC, I calculated the following number of groups for a high-SNR but unsaturated image:
ngrps = 100

Three simulations will be executed, for a short, intermediate-duration and long exposure. The number of groups will be the same for all 3, as will the number of exposures (nexp = 1). The numbers of integrations will be 1, 200 and 2000 for each case respectively.

### Steps in setting up the simulation

This notebook will go through the following steps:

* Create the scene
* Set up the simulation
* Run the simulation

Each step has its own function. Steps 1 and 2 will each write out a .ini file, which will be used as input for the final step.

In [11]:
arr = ['SLITLESSPRISM'] * 3
ngrp = [100] * 3
nints = [1, 200, 2000]


## Step 1: Creating the input scene (WASP-62)

Here we'll create the input scene for the simulations using the function wasp62_scene(). Arguments:

* scene_file: the filename for the .ini file
* write_cube: write the scene image out to a FITS file (optional; default=False)

The function returns a mirisim.skysim.scenes.CompositeSkyScene object.


In [12]:
scene_ini = wasp62_scene(scene_file='wasp62_scene.ini', write_cube=False)

2019-02-15 15:27:20,883 - INFO - Initializing Point
2019-02-15 15:27:20,887 - INFO - Initializing Background


Creating scene. Scene will be written to file wasp62_scene.ini
File wasp62_scene.ini already exists. Will be overwritten.
wasp62_scene.ini exists, overwrite (y/[n])?y


In [13]:
print(scene_ini)

wasp62_scene.ini


## Step 2: Configuring the simulation

Now I'll set up the simulations and prepare to run them. I'll set it up to loop through the 6 simulations. For this I wrote the function wasp62_sim_config. Check the docstring for descriptions and default values of the arguments. 

The function will write out another .ini file containing the simulation configuration, and it returns the output filename for further use.

In [14]:
for (a, g, i) in zip(arr, ngrp, nints):
    sim_ini = wasp62_sim_config(mode='lrs', arr=a, ngrp=g, nint=i, nexp=1, 
                   scene_file=scene_ini, out=True)
    print(sim_ini)

Found scene file wasp62_scene.ini
wasp62_SLITLESSPRISM_100G1I1E_simconfig.ini
Found scene file wasp62_scene.ini
wasp62_SLITLESSPRISM_100G200I1E_simconfig.ini
Found scene file wasp62_scene.ini
wasp62_SLITLESSPRISM_100G2000I1E_simconfig.ini


### Step 3: Run the simulation

In the following step we'll run the simulations for the 6 different cases. For each run, we need 3 input files: the scene, the simulation configuration, and the simulator setup file. The first and last of these remain the same for each run, and we loop through the list of 6 simulation config files.

After the simulation has run, the code renames the output directory to include the simulation settings to the directory.


In [10]:
cfg_files = glob.glob('*_simconfig.ini')

# configure the simulator engine - this requires no editing from the default
simulator_config = SimulatorConfig.from_default()

for f in cfg_files[:1]:
    tmp = f.split('.')
    fcomps = tmp[0].split('_')
    sim = MiriSimulation.from_configfiles(f)
    sim.run()
    outdir = sorted(glob.glob('*_*_mirisim'), key=os.path.getmtime )[-1]
    new_outdir = 'wasp62_lrstso_{0}_{1}_{2}'.format(fcomps[1], fcomps[2], outdir)
    os.rename(outdir, new_outdir)
    print(outdir, new_outdir)


2019-02-15 14:35:54,846 - INFO - Using simulation configuration: wasp62_SLITLESSPRISM_60G1I1E_simconfig.ini
2019-02-15 14:35:54,850 - INFO - Using scene configuration: wasp62_scene.ini
2019-02-15 14:35:54,857 - INFO - MIRISim version: 2.0.0
2019-02-15 14:35:54,859 - INFO - MIRI Simulation started.
2019-02-15 14:35:54,860 - INFO - Output will be saved to: 20190215_143554_mirisim
2019-02-15 14:35:54,862 - INFO - Storing configs in output directory.
2019-02-15 14:35:55,834 - INFO - Using $CDP_DIR for location of CDP files: /Users/kendrew//CDP
2019-02-15 14:35:55,994 - INFO - Reading cosmic ray properties from parameter file /Users/kendrew/anaconda3/envs/mirisim/lib/python3.5/site-packages/miri/simulators/scasim/cosmic_ray_properties.py
2019-02-15 14:35:56,003 - INFO - Reading detector properties from parameter file /Users/kendrew/anaconda3/envs/mirisim/lib/python3.5/site-packages/miri/simulators/scasim/detector_properties.py
2019-02-15 14:35:56,090 - INFO - Setting up simulated Observatio

20190215_143554_mirisim wasp62_lrstso_SLITLESSPRISM_60G1I1E_20190215_143554_mirisim
